In [1]:
import torch
import dgl
import numpy as np

d:\Miniconda3\envs\8007project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Miniconda3\envs\8007project\lib\site-packages\torchdata\datapipes\__init__.py:18: UserWarning: 
################################################################################
WARNING!
The 'datapipes', 'dataloader2' modules are deprecated and will be removed in a
future torchdata release! Please see https://github.com/pytorch/data/issues/1196
to learn more and leave feedback.
################################################################################

  deprecation_warning()


In [6]:
edge_file_path = '/root/create_graph/edge_ID.txt'
node_type_file_path = '/root/create_graph/node_type_ID.txt'

In [4]:
import pandas as pd

df=pd.read_csv('res2.csv',encoding = "ISO-8859-1")

In [5]:
df['business_id'] =  df['business_id'].astype(str)+'_POI'

In [6]:
df3=pd.read_csv("/root/autodl-tmp/my_file/res3.csv")

In [7]:
import json
f = open('/root/autodl-tmp/review.json')
data = json.load(f)
df2 = pd.DataFrame(data['data'],columns =data['columns'])

In [8]:
df2['business_id'] =  df2['business_id'].astype(str)+'_POI'

In [10]:
for i in range(len(df)):
    dic[(df.loc[i,'user_id'],df.loc[i,'business_id'])]=df.loc[i,'star']
    dic[(df.loc[i,'business_id'],df.loc[i,'user_id'])]=df.loc[i,'star']
for i in range(len(df2)):
    dic[(df2.loc[i,'user_id'],df2.loc[i,'business_id'])]=df2.loc[i,'stars']
    dic[(df2.loc[i,'business_id'],df2.loc[i,'user_id'])]=df2.loc[i,'stars']
for i in range(len(df3)):
    dic[(df3.loc[i,'user_id'],df3.loc[i,'business_id'])]=df3.loc[i,'stars']
    dic[(df3.loc[i,'business_id'],df3.loc[i,'user_id'])]=df3.loc[i,'stars']

In [10]:
import os
os.chdir('/root/create_graph')
node_type_index={} # ID: type
#两类节点，四类边
user_node_set=[]
poi_node_set=[]

with open(node_type_file_path) as f:
        lines = f.readlines()
        for line in lines:
            line_content = line.strip().split("\t")
            node = line_content[0]
            node_type = line_content[1]
            node_type_index[node] = node_type
            
            if node_type=='P':
                poi_node_set.append(node)
            elif node_type=='U':
                user_node_set.append(node)

In [11]:
os.getcwd()

'/root/create_graph'

In [12]:
user_node_set_map = { node_id: i  for i, node_id in enumerate(user_node_set) }  #对于每一类节点，dgl中都是从0 开始索引的
poi_node_set_map  = { node_id: i  for i, node_id in enumerate(poi_node_set) }

In [13]:
'''
在DGL的图里，所有的边都是有方向的， 即边 (u,v)  表示它是从节点 u  指向节点 v  的。
所以最后是有四种边
'''
count=0
user_user_edge = []
user_user_edge_weight=[]

poi_poi_edge = []
poi_poi_edge_weight=[]

user_poi_edge = []  #
user_poi_edge_weight=[]

poi_user_edge = []  #
poi_user_edge_weight=[]

with open(edge_file_path) as f:
        lines = f.readlines()
        for line in lines:
            line_content = line.strip().split("\t")
            source_node = line_content[0]
            target_node = line_content[1]
            edge_weight = eval(line_content[2])
            if node_type_index[source_node] == 'P' and node_type_index[target_node] == 'P': #POI之间的边
                poi_poi_edge.append(( poi_node_set_map[ source_node]  , poi_node_set_map[ target_node]        )  )
                poi_poi_edge.append(( poi_node_set_map[ target_node]  , poi_node_set_map[ source_node]        )  )  # 此行把 POI边变成双向的
                
                poi_poi_edge_weight.append( edge_weight)
                poi_poi_edge_weight.append( edge_weight) 
                
            elif node_type_index[source_node] == 'U' and  node_type_index[target_node] == 'U':   #user之间的边
                user_user_edge.append(( user_node_set_map[ source_node]  , user_node_set_map[ target_node]        )  )
                user_user_edge.append(( user_node_set_map[ target_node]  , user_node_set_map[ source_node]        )  ) # 此行把 User边变成双向的
                
                user_user_edge_weight.append( edge_weight)
                user_user_edge_weight.append( edge_weight)  #边重复了两次，权值自然要重复两次
            else:
                if node_type_index[source_node] == 'U':
                    user_poi_edge.append(( user_node_set_map[ source_node]  , poi_node_set_map[ target_node])  )
                    edge_weight=edge_weight*(dic[(source_node,target_node)]-1)*0.25
                        
                    user_poi_edge_weight.append( edge_weight)
                    
                    poi_user_edge.append(( poi_node_set_map[ target_node]  , user_node_set_map[ source_node]    )  )
                    poi_user_edge_weight.append( edge_weight)
                else:  #source_node P
                    poi_user_edge.append(( poi_node_set_map[ source_node]  , user_node_set_map[ target_node]    )  )
                    edge_weight=edge_weight*(dic[(source_node,target_node)]-1)*0.25
                    poi_user_edge_weight.append( edge_weight)
                    
                    user_poi_edge.append(( user_node_set_map[ target_node]  , poi_node_set_map[ source_node]        )  )
                    user_poi_edge_weight.append( edge_weight)

In [14]:
dic[('kZtl_NJjUnlIQgzFrBhXyQ_POI','-bilc2bsxQWUOjy7SIpYmQ')]

5

In [15]:
user_user_edge_array = np.array(user_user_edge)
user_user_edge_weight_array=np.array(user_user_edge_weight)#

poi_poi_edge_array = np.array(poi_poi_edge)
poi_poi_edge_weight_array =np.array(poi_poi_edge_weight)#

user_poi_edge_array  = np.array(user_poi_edge)#
user_poi_edge_weight_array  =np.array(user_poi_edge_weight)#

poi_user_edge_array  = np.array(poi_user_edge)#
poi_user_edge_weight_array  = np.array(poi_user_edge_weight)#

In [16]:
graph_data = {
    ('user', 'user_user', 'user'): (torch.tensor(user_user_edge_array[:,0]), torch.tensor(user_user_edge_array[:,1])),
    ('user', 'user_poi', 'poi'): (torch.tensor(user_poi_edge_array[:,0]), torch.tensor(user_poi_edge_array[:,1])),
    ('poi', 'poi_user', 'user'): (torch.tensor(poi_user_edge_array[:,0]), torch.tensor(poi_user_edge_array[:,1])),
    ('poi', 'poi_poi', 'poi'): (torch.tensor(poi_poi_edge_array[:,0]), torch.tensor(poi_poi_edge_array[:,1]))
}
g = dgl.heterograph(graph_data)  #构建图

In [17]:
print(g.ntypes)
print(g.metagraph().edges())
print("user_num_nodes",g.num_nodes('user'))
print("poi_num_nodes",g.num_nodes('poi'))
print(g.etypes)
print(g.canonical_etypes)

['poi', 'user']
[('poi', 'poi'), ('poi', 'user'), ('user', 'poi'), ('user', 'user')]
user_num_nodes 6379
poi_num_nodes 1434
['poi_poi', 'poi_user', 'user_poi', 'user_user']
[('poi', 'poi_poi', 'poi'), ('poi', 'poi_user', 'user'), ('user', 'user_poi', 'poi'), ('user', 'user_user', 'user')]


In [18]:
#添加边特征
g.edges['user_user'].data['w'] = torch.tensor ( user_user_edge_weight_array  , dtype=torch.float32)   
g.edges['user_poi'].data['w'] = torch.tensor( user_poi_edge_weight_array  , dtype=torch.float32)   
g.edges['poi_user'].data['w'] = torch.tensor( poi_user_edge_weight_array  , dtype=torch.float32) 
g.edges['poi_poi'].data['w'] = torch.tensor( poi_poi_edge_weight_array  , dtype=torch.float32)  

In [19]:
g

Graph(num_nodes={'poi': 1434, 'user': 6379},
      num_edges={('poi', 'poi_poi', 'poi'): 320, ('poi', 'poi_user', 'user'): 108312, ('user', 'user_poi', 'poi'): 108312, ('user', 'user_user', 'user'): 82808},
      metagraph=[('poi', 'poi', 'poi_poi'), ('poi', 'user', 'poi_user'), ('user', 'poi', 'user_poi'), ('user', 'user', 'user_user')])

In [20]:
poi_fe=np.load('/root/autodl-tmp/poi_fe.npy')
user_fe=np.load('/root/autodl-tmp/user_fe.npy')

In [21]:
g.nodes['user'].data['fe']=torch.tensor ( user_fe , dtype=torch.float32)
g.nodes['poi'].data['fe']=torch.tensor ( poi_fe , dtype=torch.float32)   

In [22]:
from dgl import save_graphs, load_graphs

In [23]:
torch.save(g, 'test')

In [24]:
m= torch.load('test')

In [25]:
m

Graph(num_nodes={'poi': 1434, 'user': 6379},
      num_edges={('poi', 'poi_poi', 'poi'): 320, ('poi', 'poi_user', 'user'): 108312, ('user', 'user_poi', 'poi'): 108312, ('user', 'user_user', 'user'): 82808},
      metagraph=[('poi', 'poi', 'poi_poi'), ('poi', 'user', 'poi_user'), ('user', 'poi', 'user_poi'), ('user', 'user', 'user_user')])